In [6]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from EnergySaverLLM.Agent import ChargingAgent, _run_with_exec, _insert_params
from flaml.autogen.code_utils import extract_code

In [8]:
# global params_filepath
params_filepath = "EnergySaverLLM/Model/params/EVCharging.json"

In [9]:
with open(params_filepath) as f:
    json_str = f.read()

In [10]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-3.5-turbo"],
        # "model": ["gpt-3.5-turbo"],
    },
)

In [11]:
autogen.oai.ChatCompletion.start_logging()

In [12]:
code_path = "EnergySaverLLM/Model/EVCharging.py"

In [13]:
with open(code_path) as f:
    code = f.read()

In [14]:
# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from datetime import datetime
from gurobipy import GRB, Model
import json

# Fetching data from JSON file

params_filepath = "EnergySaverLLM/Model/params/EVCharging.json"

stored_params = json.load(open(params_filepath))

.
.
.

    values = model.getAttr("X", all_vars)
    names = model.getAttr("VarName", all_vars)

    for i, time_str in enumerate(keys):
        if(values[i]>0):
            print(f"Scheduled consumption at Hour {time_str} : {int(values[i]):d} KWH")

else:
    print("Not solved to optimality. Optimization status:", model.status)



In [15]:
example_qa = """
----------
Instruction: Charge the car till 9 AM.
Answer Code:
```JSON
"end_charge_time": 9
```

----------
Question: Charge the car to full charge by 9 AM
Answer Code:
```JSON
"end_charge": 1.00,
"end_charge_time": 9
```
"""

In [16]:
%%capture
agent = ChargingAgent(
    name="Tesla Charging Example",
    source_code=code,
    debug_times=1,
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [17]:
user.initiate_chat(agent, message="Increase Electricity cost by 30%")

user (to Tesla Charging Example):

Increase Electricity cost by 30%

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
{
    "elec_cost": {
        "0": 0.39,
        "1": 0.39,
        "2": 0.39,
        "3": 0.39,
        "4": 0.39,
        "5": 0.39,
        "6": 0.39,
        "7": 0.455,
        "8": 0.455,
        "9": 0.455,
        "10": 0.455,
        "11": 0.39,
        "12": 0.39,
        "13": 0.39,
        "14": 0.39,
        "15": 0.39,
        "16": 0.39,
        "17": 0.39,
        "18": 0.39,
        "19": 0.455,
        "20": 0.455,
        "21": 0.455,
        "22": 0.39,
        "23": 0.39
    }
}
```

--------------------------------------------------------------------------------
{'0': 0.39, '1': 0.39, '2': 0.39, '3': 0.39, '4': 0.39, '5': 0.39, '6': 0.39, '7

In [18]:
user.initiate_chat(agent, message="Complete charging battery to 100% and reduce carbon cost by 10%")

user (to Tesla Charging Example):

Complete charging battery to 100% and reduce carbon cost by 10%

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
{
  "end_charge": 1.0,
  "carbon_cost": {
        "0": 22.5,
        "1": 22.5,
        "2": 22.5,
        "3": 22.5,
        "4": 22.5,
        "5": 22.5,
        "6": 22.5,
        "7": 22.5,
        "8": 22.5,
        "9": 22.5,
        "10": 22.5,
        "11": 22.5,
        "12": 22.5,
        "13": 22.5,
        "14": 22.5,
        "15": 22.5,
        "16": 22.5,
        "17": 22.5,
        "18": 22.5,
        "19": 22.5,
        "20": 22.5,
        "21": 22.5,
        "22": 22.5,
        "23": 22.5
    }
}
```

--------------------------------------------------------------------------------
{'0': 0.39, '1': 0.39, '2': 0.39, '

In [12]:
user.initiate_chat(agent, message="Complete charging battery to 100% by 9 AM tomorrow")

user (to Tesla Charging Example):

Complete charging battery to 100% by 9 AM tomorrow

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

"end_charge": 1.0,
"end_charge_time": 9

--------------------------------------------------------------------------------
{'0': 0.3, '1': 0.3, '2': 0.3, '3': 0.3, '4': 0.3, '5': 0.3, '6': 0.3, '7': 0.35, '8': 0.35, '9': 0.35, '10': 0.35, '11': 0.3, '12': 0.3, '13': 0.3, '14': 0.3, '15': 0.3, '16': 0.3, '17': 0.3, '18': 0.3, '19': 0.35, '20': 0.35, '21': 0.35, '22': 0.3, '23': 0.3}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 48 columns and 96 nonzeros
Model fingerprint: 0x7b4bbd5a
Variable types: 0 continuous, 

In [14]:
user.initiate_chat(agent, message="Complete charging battery by 9 PM")

user (to Tesla Charging Example):

Complete charging battery to 100% by 9 AM tomorrow

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
end_charge_level = 1
end_time_H = 9
this_charge_KWH = (end_charge_level - start_charge_level) * total_battery_capacity
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
end_charge_level = 1
end_time_H = 9
this_charge_KWH = (end_charge_level - start_charge_level) * total_battery_capacity
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)

--- One-Word Answer: SAFE or DANGER ---


----------------------------------------------------------------

In [17]:
user.initiate_chat(agent, message="Reduce max allowed electricity cost to 15 dollars")

user (to Tesla Charging Example):

Reduce max allowed electricity cost to 15 dollars

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
max_elec_cost = 15
model.addConstr(sum(x[i] * elec_costs[i] for i in keys) <= max_elec_cost)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
max_elec_cost = 15
model.addConstr(sum(x[i] * elec_costs[i] for i in keys) <= max_elec_cost)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Tesla Charging Example):

SAFE

--------------------------------------------------------------------------------
{'0': 0.3, '1': 0.3, '2': 0.3, '3': 0.3, '4': 0.3, '5': 0.3, '6': 0.3, '7':

In [18]:
user.initiate_chat(agent, message="Charge the car to full charge by 6 AM tomorrow")

user (to Tesla Charging Example):

Charge the car to full charge by 6 AM tomorrow

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
end_charge_level = 1
this_charge_KWH = (end_charge_level-start_charge_level)*total_battery_capacity
model.addConstr(sum(x[i] for i in keys) == this_charge_KWH)
end_time_H = 30
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
end_charge_level = 1
this_charge_KWH = (end_charge_level-start_charge_level)*total_battery_capacity
model.addConstr(sum(x[i] for i in keys) == this_charge_KWH)
end_time_H = 30
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_

In [20]:
user.initiate_chat(agent, message="Charge the car by 9 AM tomorrow")

user (to Tesla Charging Example):

Charge the car by 9 AM tomorrow

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
end_time_H = 33
upper_bounds = [0]*len(elec_costs)
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
end_time_H = 33
upper_bounds = [0]*len(elec_costs)
upper_bounds[curr_time_H:end_time_H] = [max_power_KW] * (end_time_H - curr_time_H)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Tesla Charging Example):

SAFE

--------------------------------------------------------------------------------
{'0': 0.3, 